Now we want to add in some quantum features. Essentially what we will do is add some more interactions (super/sub-diagonals) such that the matrix (Hamiltonian) is no longer diagonal in the computation basis. In general this will make finding the lowest energy state very hard! The model we will study is called the *transverse field quantum Ising model*:

$$ \hat{H} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^z \hat{\sigma}_j^z - h\sum_i \hat{\sigma}_i^x $$

Now, instead of just being *numbers* the $\hat{\sigma}$ are 2x2 *matrices*. I've neglected to write a bunch of outer products with the identity. What's going on is that $\hat{\sigma}_x$ acts as a "flipper", taking up spins to down and vice versa:

$$ \hat{\sigma}^x\left| 1 \right\rangle = \left| 0 \right\rangle $$

and

$$ \hat{\sigma}^x\left| 0 \right\rangle = \left| 1 \right\rangle $$

Clearly, the addition of a bunch of these means our resulting $\hat{H}$ above is *not* diagonal in the simluation basis, at least as long as $h$ is nonzero. (If you need to convince yourself, write out the full 4x4 $\hat{H}$ for a system with just 2 sites.)

$h$ controls how likely it is we will flip a particular site. We will use our simulation to investigate its effects. It's easy to write code to *generate* this matrix:

In [1]:
σᶻ = [1 0; 0 -1]

2×2 Array{Int64,2}:
 1   0
 0  -1

In [2]:
σˣ = [0 1; 1 0]

2×2 Array{Int64,2}:
 0  1
 1  0

In [3]:
σˣ * [1,0]

2-element Array{Int64,1}:
 0
 1

In [4]:
σˣ * [0, 1]

2-element Array{Int64,1}:
 1
 0

In [5]:
kron(σᶻ,σᶻ) # this is σᶻᵢ σᶻⱼ

4×4 Array{Int64,2}:
 1   0   0  0
 0  -1   0  0
 0   0  -1  0
 0   0   0  1

In [8]:
# An example of a 4 site Hamiltonian

H = -kron(kron(kron(σᶻ,σᶻ), eye(2)), eye(2)) - kron(eye(2), kron(kron(σᶻ,σᶻ), eye(2))) - kron(eye(2), kron(eye(2),kron(σᶻ,σᶻ)))
H -= 2*(kron(kron(kron(σˣ, eye(2)), eye(2)), eye(2)) + kron(eye(2), kron(kron(σˣ, eye(2)), eye(2))) + kron(eye(2), kron(eye(2), kron(σˣ, eye(2)))))

16×16 Array{Float64,2}:
 -3.0  -0.0  -2.0  -0.0  -2.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -1.0  -0.0  -2.0  -0.0  -2.0     -0.0  -0.0  -0.0   0.0  -0.0  -0.0
 -2.0  -0.0   1.0   0.0  -0.0  -0.0     -2.0   0.0  -0.0  -0.0   0.0  -0.0
 -0.0  -2.0   0.0  -1.0  -0.0  -0.0      0.0  -2.0  -0.0  -0.0  -0.0  -0.0
 -2.0  -0.0  -0.0  -0.0   1.0   0.0     -0.0  -0.0  -2.0   0.0  -0.0  -0.0
 -0.0  -2.0  -0.0  -0.0   0.0   3.0  …  -0.0  -0.0   0.0  -2.0  -0.0  -0.0
 -0.0  -0.0  -2.0  -0.0  -2.0   0.0      0.0  -0.0  -0.0  -0.0  -2.0  -0.0
 -0.0  -0.0  -0.0  -2.0   0.0  -2.0     -0.0  -0.0  -0.0  -0.0  -0.0  -2.0
 -2.0  -0.0  -0.0  -0.0  -0.0  -0.0     -2.0   0.0  -2.0  -0.0  -0.0  -0.0
 -0.0  -2.0  -0.0  -0.0  -0.0   0.0      0.0  -2.0  -0.0  -2.0  -0.0  -0.0
 -0.0  -0.0  -2.0   0.0  -0.0  -0.0  …   3.0   0.0  -0.0  -0.0  -2.0  -0.0
 -0.0  -0.0   0.0  -2.0  -0.0  -0.0      0.0   1.0  -0.0  -0.0  -0.0  -2.0
 -0.0  -0.0  -0.0  -0.0  -2.0   0.0     -0.0  -0.0  -1.0   0.0  -2.0  -0.0
 

Gross! We can save on storage space by using Julia's native `BitArray` type. We can even build our basis the naive way and convert it at some up-front performance cost. Or, we can build the list of `BitArray`s ourselves:

In [1]:
function bit_rep(element::Integer, L::Integer)
    bit_rep = falses(L)
    for site in 1:L
       bit_rep[site] = (element >> (site - 1)) & 1
    end
    return bit_rep
end

function int_rep(element::BitVector, L::Integer)
    int = 1
    for site in 1:L
       int += (element[site] << (site - 1))
    end
    return int
end

function generate_basis(L::Integer)
    basis = fill(falses(L), 2^L)
    for elem in 1:2^L
        basis[elem] = bit_rep(elem, L)
    end
    return basis
end

function TransverseFieldIsing(L::Integer, h::Real=0.)
    basis = generate_basis(L)
    H = zeros(2^L, 2^L)
    for (index, element) in enumerate(basis)
        # the diagonal part is easy
        diag_term = 0.
        for site in 1:L-1
            diag_term -= !xor(element[site], element[site+1])
        end
        H[index, index] = diag_term
        # off diagonal part
        for site in 1:L-1
            mask = falses(L)
            mask[site] = true
            new_element = xor.(element, mask)
            new_index = int_rep(new_element, L)
            H[index, new_index] = -h
        end
    end
    return Hermitian(H), basis
end

TransverseFieldIsing (generic function with 2 methods)

Some things to note:
- This matrix is very sparse! That's pretty generic to quantum systems, and it's a mixed blessing.
- For larger systems, it may make sense to use the Julia sparse matrix types (we'll see how to do this shortly).
- For now we have set $h$ to be the same everywhere. We aren't *forced* to do this - we could introduce *disorder* and see more interesting physics. This is a good **exercise** for interested non-physicists.

We can still use Julia's linear algebra methods like `eigfact`, of course, they'll just be slower:

In [2]:
H, basis = TransverseFieldIsing(8, 1.)
@time eigfact(H)

  0.358076 seconds (72.14 k allocations: 5.631 MiB, 24.65% gc time)


Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-10.1634, -9.97258, -8.85079, -8.57048, -8.50815, -8.35801, -8.09964, -8.09511, -7.78885, -7.73286  …  2.67879, 2.74717, 2.80552, 2.92148, 3.25234, 3.27357, 3.38151, 3.41845, 3.49801, 3.5997], [-0.0141714 -0.066122 … -0.0914145 0.0254545; -0.0212904 -0.0997415 … 0.018321 0.00453354; … ; -0.0782906 0.0249041 … 0.0370397 0.0368878; -0.18211 0.0647595 … -0.00520433 -0.0109567])

## Dude, Where's My Parallelism?

For many interesting physical problems (one of which we're about to see) we *don't need* to use multiple nodes anymore. This is good - parallel computing is really cool but it's also really difficult to do well! The promised parallelism is coming! But we should all be happy we can practically look at (some) many-body physics with doing many-body computing (yet).

## It's Just a Phase

Let's vary $h$ and see what happens. Since we're looking at *quantum magnets* we will compute the *overall magnetization*. This quantity is:

$$ M = \frac{1}{N}\sum_{i} \sigma^z_i $$

where $\sigma^z_i$ is the value of the spin on site $i$ when we measure. If $M$ is 0 there is no overall magnetic moment. We divide by the number of sites so that we can compare results for various systems. Since we're using `0` to represent spin down ($\sigma^z = -1$), and `1` to represent spin up ($\sigma^z = +1$), in our code this will look like:

In [24]:
function magnetization(state::Vector, basis)::Float64
    M = 0.
    for (index, element) in enumerate(basis)
        element_M = 0.
        for spin in element
            element_M += (state[index]^2 * (spin ? 1 : -1))/length(element)
        end
        @assert abs(element_M) <= 1
        M += abs(element_M)
    end
    return M
end

magnetization (generic function with 2 methods)

Now we would like to examine the effects of $h$. We will:
  1. Find a variety of $h$ to look at.
  2. For each, compute the *lowest energy eigenvector* (groundstate) of the corresponding Hamiltonian.
  3. For each groundstate, compute the overall magnetization $M$.
  4. Plot $M(h)$ for a variety of system sizes, and see if anything cool happens.

In [25]:
using Plots
plotly()
plot(fmt=:png)
hs = logspace(-2., 2., 10)
Ls = [4, 6, 8, 10, 12]
for L in Ls
    M = zeros(hs)
    for (i,h) in enumerate(hs)
        H, basis = TransverseFieldIsing(8, h)
        vals, vecs = eig(H)
        groundstate = vecs[:,1]
        M[i] = magnetization(groundstate, basis)
    end
    plot!(hs, M, label="L=$L")
end
plot!(xscale=:log)
gui()

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:64
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:51
 [3] indices at ./abstractarray.jl:57 [inlined]
 [4] indices1 at ./abstractarray.jl:64 [inlined]
 [5] linearindices at ./abstractarray.jl:92 [inlined]
 [6] endof at ./abstractarray.jl:127 [inlined]
 [7] optimize_ticks_typed(::Float64, ::Float64, ::Bool, ::Array{Tuple{Float64,Float64},1}, ::Int64, ::Int64, ::Int64, ::Float64, ::Float64, ::Float64, ::Float64, ::Bool, ::Void) at /Users/kshyatt/.julia/v0.6/PlotUtils/src/ticks.jl:162
 [8] (::PlotUtils.#kw##optimize_ticks)(::Array{Any,1}, ::PlotUtils.#optimize_ticks, ::Float64, ::Float64) at ./<missing>:0
 [9] optimal_ticks_and_labels(::Plots.Axis, ::Void) at /Users/kshyatt/.julia/v0.6/Plots/src/axes.jl:185
 [10] get_ticks(::Plots.Axis) at /Users/kshyatt/.julia/v0.6/Plots/src/axes.jl:228
 [11] tick_padding(::Plots.Axis) at /Users/kshyatt/.julia/v0.6/Plots/src/backends.jl:71
 [12] _update_min_padding!(::Plots.Subplot{P

## Exercise

So far we have seen functions to compute the energy, and compute and plot the magnetization. There are many other physically interesting quantities we could plot! Try plotting the [magnetic susceptibility](https://en.wikipedia.org/wiki/Magnetic_susceptibility) - how does it vary across the transition?